In [15]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [17]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [18]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RkZXMuxnKDp0xraOh75IChu9AG356Y7jp8BoVrGxwMEL',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-y3vcdpcbkrumz5'
object_key = 'dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [19]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [20]:
x_train = train_datagen.flow_from_directory(r'dataset/train',
                                            target_size=(64, 64),
                                            batch_size=5,
                                            color_mode='grayscale',
                                            class_mode='categorical')

x_test = test_datagen.flow_from_directory(r'dataset/test',
                                          target_size=(64, 64),
                                          batch_size=5,
                                          color_mode='grayscale',
                                          class_mode='categorical') 


Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [21]:
print(x_train.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [22]:
model=Sequential()

In [23]:
model.add(Conv2D(32,(3,3),input_shape=(64,64,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


In [24]:
model.add(Flatten())

In [25]:
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=6,activation='softmax'))

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [27]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:

model.fit_generator(x_train,
                    steps_per_epoch = 119 , 
                    epochs = 20, 
                    validation_data = x_test,
                    validation_steps = 6 )

/tmp/wsuser/ipykernel_164/2938001502.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,


Epoch 1/20
119/119 [==============================] - 6s 40ms/step - loss: 1.5133 - accuracy: 0.3687 - val_loss: 0.7563 - val_accuracy: 0.7667
Epoch 2/20
119/119 [==============================] - 5s 38ms/step - loss: 0.7680 - accuracy: 0.7003 - val_loss: 0.7207 - val_accuracy: 0.7667
Epoch 3/20
119/119 [==============================] - 5s 38ms/step - loss: 0.5414 - accuracy: 0.7795 - val_loss: 0.4453 - val_accuracy: 0.8333
Epoch 4/20
119/119 [==============================] - 5s 40ms/step - loss: 0.3630 - accuracy: 0.8653 - val_loss: 0.3510 - val_accuracy: 0.9000
Epoch 5/20
119/119 [==============================] - 5s 38ms/step - loss: 0.3043 - accuracy: 0.8923 - val_loss: 0.4982 - val_accuracy: 0.8667
Epoch 6/20
119/119 [==============================] - 4s 37ms/step - loss: 0.2792 - accuracy: 0.8990 - val_loss: 0.4222 - val_accuracy: 0.8667
Epoch 7/20
119/119 [==============================] - 5s 40ms/step - loss: 0.2015 - accuracy: 0.9360 - val_loss: 0.3102 - val_accuracy: 0.9000

In [29]:
model.save('gesture.h5')


In [45]:
!tar -zcvf image-classification-model_new.tgz gesture.h5

gesture.h5


In [46]:
ls -1

dataset/
gesture.h5
image-classification-model_new.tgz


In [47]:
!pip install watson-machine-learning-client --upgrade

In [80]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"ARfcmDqbH5jx8YhSBmTBtKJQH820oh39jr17z0ueJKFB"
    
}
client=APIClient(wml_credentials)

In [81]:
client=APIClient(wml_credentials)

In [82]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [83]:
space_uid=guid_from_space_name(client,'imageclassification')
print("Space UID = " +space_uid)

Space UID = f85403b9-6756-41b8-94c1-353d912fc0a7


In [84]:
client.set.default_space(space_uid)

'SUCCESS'

In [85]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [88]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [89]:
model_details=client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [90]:
print(model_id)

c507de1d-99bd-4597-a76b-9cd42ebcc101


In [91]:
client.repository.download(model_id,'gestureModel.tar.gz')

Successfully saved model content to file: 'gestureModel.tar.gz'


'/home/wsuser/work/gestureModel.tar.gz'

In [95]:
from keras.models import load_model
from keras.preprocessing import image

In [96]:
model=load_model("gesture.h5")

In [112]:
import keras
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RkZXMuxnKDp0xraOh75IChu9AG356Y7jp8BoVrGxwMEL',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-y3vcdpcbkrumz5'
object_key = '2.jpg'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']



In [114]:
img=keras.preprocessing.image.load_img(streaming_body_3,target_size=(64,64))

TypeError: expected str, bytes or os.PathLike object, not StreamingBody